# Gather & Wrangle no-show data
***
Get & clean the data from the no-show to doctor appointments dataset from Kaggle, containing information on doctor visits at a clinic in Brazil (?) for several months in 2016, noting if the patient showed up to the scheduled appointment or not.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
import io
import requests
import zipfile

## 1. Gather Data
***
Data is downloaded from Kaggle: https://www.kaggle.com/joniarroba/noshowappointments

#### 1.1 programmatically download from Kaggle 

In [3]:
# make a data-folder
data_folder = 'data'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

In [4]:
# get the url (inspect api call made when clicking download)
url = 'https://storage.googleapis.com/kaggle-data-sets/792%2F3538%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1595062432&Signature=K1D%2FGYMPDHgZphrKA8CICaNN2B5jGlH%2Bh6%2BgvYKGr1fYI%2FthrxnM2fcSp7yYfx6fP70g%2BCp7SjegMtwxAWh9W0OzoLuONQsL7uj5PWs%2F85hxHM9zlDcBQw6j6weWp5imvFawQePVzktzytJTSpu3N1V1ACwYmnHLy%2B%2FLZwJSBNhNiuiqbw%2BHeYdiyfXAlj7GGZqTi0z7rPVW8c3fh6OarnMtidTmmwpJvCnYySfz8VVKvFp%2FgRPzo%2FbHS59VXBqCIgGC7cZVMQ0DfWSDOsrsTWAJv4vII0rTJE6MAv1AdCKEMFrMkplPUZQF4z0m5LUWJFF8dO1EoUXPt5NDQrF9zg%3D%3D'

# download
r = requests.get(url, stream=True)
with zipfile.ZipFile(io.BytesIO(r.content)) as myzip:
    output_file = myzip.namelist()[0]
    myzip.extract(output_file, data_folder)

#### 1.2 Load & Inspect

In [5]:
df = pd.read_csv(data_folder + '/' + output_file)
print(df.shape)
df.head()

(110527, 14)


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


> We seem to have the correct data.

## 2. Assessment
***

#### 2.1 data-types

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


#### 2.2 duplicates/missing values

In [7]:
df.duplicated().sum()

0

In [8]:
df.isnull().sum()

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

#### 2.3 value ranges

In [9]:
df.nunique()

PatientId          62299
AppointmentID     110527
Gender                 2
ScheduledDay      103549
AppointmentDay        27
Age                  104
Neighbourhood         81
Scholarship            2
Hipertension           2
Diabetes               2
Alcoholism             2
Handcap                5
SMS_received           2
No-show                2
dtype: int64

In [10]:
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [11]:
df['ScheduledDay'].agg([min, max])

min    2015-11-10T07:13:56Z
max    2016-06-08T20:07:23Z
Name: ScheduledDay, dtype: object

In [12]:
df['AppointmentDay'].agg([min, max])

min    2016-04-29T00:00:00Z
max    2016-06-08T00:00:00Z
Name: AppointmentDay, dtype: object

#### 2.4 summary observations
No further action:
* no missing data, no duplicates
* gender 

Action required:
1. column naming in inconsistent (`PatientId`, `AppointmentID`, `SMS_received`), we'll convert it to lowercase with underscores to make our life a bit easier.
2. `age` values range between -1 and 115, where -1 is an evident issue (perhaps this was a missing data).
3. `handicap` ranges between 0 and 4, whereas most other values are binary. Not sure what the meaning is (no info with the dataset either). We'll keep it as categorical value, but also add a binary column (0 or >0).
4. `ScheduledDay` and `AppointmentDay` are stored as string, we'll convert this to datetime (and date for appointmentday as there is no time available). Range seems okay, minimum `ScheduledDay` at 2015-11-10 is a bit far from the appointment range, but not unreasonable.
5. Convert `PatientId` to string, this makes it fully available to read in the dataframe (not cut to scientific notation).

Once we get to create a model, we will furthermore need dummy variables for Gender (now M/F). Most other variables (i.e. diabetes) are already binary.


#### 2.5 check implications of our changes
Briefly check 

In [13]:
# how often negative age?
df[df['Age']<0]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
99832,4.659432e+14,5775010,F,2016-06-06T08:58:13Z,2016-06-06T00:00:00Z,-1,ROMÃO,0,0,0,0,0,0,No


In [14]:
# how often age >100?
df[df['Age']>100]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
58014,9.762948e+14,5651757,F,2016-05-03T09:14:53Z,2016-05-03T00:00:00Z,102,CONQUISTA,0,0,0,0,0,0,No
63912,3.196321e+13,5700278,F,2016-05-16T09:17:44Z,2016-05-19T00:00:00Z,115,ANDORINHAS,0,0,0,0,1,0,Yes
63915,3.196321e+13,5700279,F,2016-05-16T09:17:44Z,2016-05-19T00:00:00Z,115,ANDORINHAS,0,0,0,0,1,0,Yes
68127,3.196321e+13,5562812,F,2016-04-08T14:29:17Z,2016-05-16T00:00:00Z,115,ANDORINHAS,0,0,0,0,1,0,Yes
76284,3.196321e+13,5744037,F,2016-05-30T09:44:51Z,2016-05-30T00:00:00Z,115,ANDORINHAS,0,0,0,0,1,0,No
90372,2.342836e+11,5751563,F,2016-05-31T10:19:49Z,2016-06-02T00:00:00Z,102,MARIA ORTIZ,0,0,0,0,0,0,No
97666,7.482346e+14,5717451,F,2016-05-19T07:57:56Z,2016-06-03T00:00:00Z,115,SÃO JOSÉ,0,1,0,0,0,1,No


> Stratefy outlined above seems okay. We'll remove the case with the lower age and keep the higher age.

## 3. Clean Data
***
We'll address the issues found in section 2.

In [15]:
df_clean = df.copy()

#### 3.1 rename columns

In [16]:
[col.lower() for col in df_clean.columns]

['patientid',
 'appointmentid',
 'gender',
 'scheduledday',
 'appointmentday',
 'age',
 'neighbourhood',
 'scholarship',
 'hipertension',
 'diabetes',
 'alcoholism',
 'handcap',
 'sms_received',
 'no-show']

In [17]:
# not that many - quick to manually assign
df_clean.columns = ['patient_id',
                    'appointment_id',
                    'gender',
                    'scheduled_day',
                    'appointment_day',
                    'age',
                    'neighbourhood',
                    'scholarship',
                    'hipertension',
                    'diabetes',
                    'alcoholism',
                    'handicap',
                    'sms_received',
                    'no_show']

In [18]:
df_clean.columns

Index(['patient_id', 'appointment_id', 'gender', 'scheduled_day',
       'appointment_day', 'age', 'neighbourhood', 'scholarship',
       'hipertension', 'diabetes', 'alcoholism', 'handicap', 'sms_received',
       'no_show'],
      dtype='object')

#### 3.2 remove invalid age

In [19]:
df_clean = df_clean[df_clean['age'] >= 0]
assert len(df_clean[df_clean['age'] < 0]) == 0

#### 3.3 update 'handicap' column

In [20]:
df_clean['handicap'].value_counts()

0    108285
1      2042
2       183
3        13
4         3
Name: handicap, dtype: int64

In [25]:
# binary columns
df_clean['handicap_bin'] = (df_clean['handicap'] > 0)*1
df_clean['handicap_bin'].head()

0    0
1    0
2    0
3    0
4    0
Name: handicap_bin, dtype: int64

In [29]:
assert df_clean['handicap_bin'].sum() == 2042 + 183 + 13 + 3

In [26]:
# categorical
df_clean['handicap'] = df_clean['handicap'].astype('category')
df_clean['handicap'].head()

0    0
1    0
2    0
3    0
4    0
Name: handicap, dtype: category
Categories (5, int64): [0, 1, 2, 3, 4]

In [30]:
df_clean['handicap'].value_counts()

0    108285
1      2042
2       183
3        13
4         3
Name: handicap, dtype: int64

#### 3.4 ScheduledDay and AppointmentDay as date(time)

In [31]:
df_clean['scheduled_day'] = pd.to_datetime(df_clean['scheduled_day']).dt.tz_localize(None)
df_clean['scheduled_day'].head()

0   2016-04-29 18:38:08
1   2016-04-29 16:08:27
2   2016-04-29 16:19:04
3   2016-04-29 17:29:31
4   2016-04-29 16:07:23
Name: scheduled_day, dtype: datetime64[ns]

In [32]:
df_clean['appointment_day'] = pd.to_datetime(df_clean['appointment_day']).dt.tz_localize(None)
df_clean['appointment_day'].head()

0   2016-04-29
1   2016-04-29
2   2016-04-29
3   2016-04-29
4   2016-04-29
Name: appointment_day, dtype: datetime64[ns]

#### 3.5 convert `patient_id` to a string

In [33]:
df_clean['patient_id'] = df_clean['patient_id'].astype(int).astype(str)
df_clean['patient_id'].head()

0     29872499824296
1    558997776694438
2      4262962299951
3       867951213174
4      8841186448183
Name: patient_id, dtype: object

#### 3.6 final check

In [34]:
df_clean.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,sms_received,no_show,handicap_bin
0,29872499824296,5642903,F,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No,0
1,558997776694438,5642503,M,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No,0
2,4262962299951,5642549,F,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No,0
3,867951213174,5642828,F,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,0
4,8841186448183,5642494,F,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No,0


In [35]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110526 entries, 0 to 110526
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   patient_id       110526 non-null  object        
 1   appointment_id   110526 non-null  int64         
 2   gender           110526 non-null  object        
 3   scheduled_day    110526 non-null  datetime64[ns]
 4   appointment_day  110526 non-null  datetime64[ns]
 5   age              110526 non-null  int64         
 6   neighbourhood    110526 non-null  object        
 7   scholarship      110526 non-null  int64         
 8   hipertension     110526 non-null  int64         
 9   diabetes         110526 non-null  int64         
 10  alcoholism       110526 non-null  int64         
 11  handicap         110526 non-null  category      
 12  sms_received     110526 non-null  int64         
 13  no_show          110526 non-null  object        
 14  handicap_bin     110

## 4. Store cleaned data
*** 
We'll store this data in a hdf file to keep our formatting intact. This allows us to split processing the data over multiple notebooks, improving the structure.

In [51]:
filename = data_folder + '/' + 'no_shows_clean.h5'
print(filename)

data/no_shows_clean.h5


In [48]:
df_clean.to_hdf(filename, key = 'no_shows', format = 'table')

In [50]:
df_check = pd.read_hdf(filename)
print(df_check.shape)
df.info()

(110526, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB
